# READ BEFORE RUNNING

Required installation

ffmpeg (https://github.com/kkroening/ffmpeg-python) - to install type "pip install ffmpeg" in command line.

pydub (https://github.com/jiaaro/pydub#installation) - to install type "pip install pydub" in command line.

thinkx (thinkdsp) (https://pypi.org/project/thinkx/) - to install type "pip install thinkx" in command line.

In [1]:
import thinkdsp
import numpy as np
import math
from thinkdsp import play_wave
from pydub import AudioSegment


In [2]:
#synth
class synthesizer:
    #creates a simple sin wave
    def sin_create(f,time):
        return(np.sin(2*np.pi*f*time))
    #creates envelope according to adsr
    def envelope (A,D,D_A,S,R,samp):
          # y= 1/A*x
        attack_ = np.linspace(0,1,int(samp*A))
        decay_ = np.linspace(1,1-D_A,int(samp*D))
        sustain_ = np.empty(int(samp*S))
        sustain_.fill(1-D_A)
        release_ = np.linspace(1-D_A,0,int(R*samp))
        env_ = np.concatenate((attack_,decay_,sustain_,release_))
        return(env_)
    # creates kick from transient envelopes and initial frequency
    def kick(f,trans,trans_pitch,time,intercept):
        return(trans*np.sin(2*np.pi*((f*(1+100000**(-trans_pitch+intercept)))*time)))
    #shifts the frequency according to major chords
    def single_shift(f,key_shift):
        if key_shift == 0:
#             f_list.append(f*4/3)
            new_wav = f  
        elif key_shift == -1:
            new_wav = 8/9*f
        elif key_shift == -2:
            new_wav = 4/5*f
        elif key_shift == -3:
            new_wav = 3/4*f
        elif key_shift == -4:
            new_wav = 2/3*f
        elif key_shift == -5:
            new_wav = 3/5*f
        elif key_shift == -6:
            new_wav = 3/5*f
        elif key_shift == 1:
#             f_list.append(f*4/3)
            new_wav = 9/8*f  
        elif key_shift == 2:
#             f_list.append(f*4/3)
            new_wav = 5/4*f    
        elif key_shift == 3:
#             f_list.append(f*3/2)
            new_wav = 4/3*f
        elif key_shift == 4:
#             f_list.append(f*3/2)
            new_wav = 3/2*f
        elif key_shift == 5:
            new_wav = 5/3*f
        elif key_shift == 6:
            new_wav = 5/3*f
        else:
            new_wav = 0 
        return(new_wav)
    #creates unison effect
    def uni_(f,voices, detune_lim,time):
        detune = np.linspace(-1*detune_lim,1*detune_lim, voices)
        chorus_ = np.sin(2*np.pi*f*time)
        for i in range(0,voices):
            chorus_ = chorus_ + np.sin(2*np.pi*(f+detune[i])*time)
        return(chorus_)
    #creates frequency modulation of fundamentla frequency based on fm matrix.
    def fm_(ff,fm, A ,time):
#         for i in range(0,len(fm)):
#             for j in range(0,len(fm[i]):
#                 jj = len(fm[i])-j
#                 f[i,j] =np.sin(2*np.pi*(fc+ A[i][jj]*fm[i][jj])*time)
#                 fc = fm[i][j]
#             fc = ff
        tpi_ = 2*np.pi*time
        #equation for FM modulation for 6 modulators in series
        return(np.sin((ff+A[0]*np.sin((fm[0]+A[1]*np.sin((fm[1]+A[2]*np.sin((fm[2]+A[3]*np.sin((fm[3]+A[4]*np.sin((fm[4]+A[5]*np.sin((fm[5])*tpi_))*tpi_))*tpi_))*tpi_))*tpi_))*tpi_))*tpi_))
    # turns np to wave
    def wave_map(wave, samp):
        return(thinkdsp.Wave(wave, framerate = 44100))

    

In [3]:
#Sequencer
class sequencer:
    def time_shift(wave,t_shift):
        wav = wave.copy()
        wav.shift(t_shift)
        return(wav)
    def time_np(t,samp):
        return(np.linspace(0,t, int(t*samp)))

In [4]:
nyquist = 44100

#________song_1_____________________
 
f1 = 500
# Keys = [0,-4,2,9,1,3,10]
# time_shift = [0,0,.5,1,2,2.5,3]
# time_list = [1,1,1,2,1,1,2]

#keys that will be played
Keys = [-7,-2 ,8,6,1,-2,-1,10,-5,-1,8,6,3,-2,9,10,-4,-5,-2,9,7,4]
# time that each key will be played
time_shift = [0,.3,.6,.92,1.23,1.69,2,2.3,2.61,3.07,3.38,3.69,4.15,4.46,4.76,5.07,5.53,6,6.3,6.76,7.23,7.53]
#how long each key lasts
time_list = [1,.4,1,2,.5,1,.3,1,.5,1,2,2,.5,1,1,1,2,3,1,.5,.5,.5]

#Unison speicifcations
#how many of the same copies of the wave we want
voices = 5
# pitch (Freq) variation
detune_lim = 4

for i in range(0, len(Keys)):
#     if Keys[i] > abs(6):
#         k = Keys[i]//7
#         fshift = f1*2**k
#         r = Keys[i]%7
#     else:
#         r = Keys[i]
#         fshift = f1
    k = Keys[i]//7
    fshift = f1*2**k
    r = Keys[i]%7
    #shifts the fundamental frequency f1 to a given key
    ff = synthesizer.single_shift(fshift,r)
    #maps the time
    time = sequencer.time_np(time_list[i], nyquist)
    #creates the unison
    c = synthesizer.uni_(ff,voices,detune_lim,time)
    #transforms it into a wave
    c = synthesizer.wave_map(c, nyquist)
    #sets when the key is played occurding to time_shifts
    c = sequencer.time_shift(c,time_shift[i])
    if i == 0:
        song_1 = c
    else:
        #adds the keys into the song
        song_1 = song_1 + c
#cumulative product of all of the keys
song_1.write("song_1.wav")

sound = AudioSegment.from_file("song_1.wav", format='wav', frame_rate=22050)
sound = sound.set_frame_rate(22050)
sound.export("song_1.wav", format='wav')

Writing song_1.wav


C:\Users\Jason\anaconda3\lib\site-packages\thinkdsp.py:1115: UserWarning: Warning: normalizing before quantizing.
  warnings.warn('Warning: normalizing before quantizing.')


<_io.BufferedRandom name='song_1.wav'>

In [5]:
#_____________song_2___________________
#sets the BPM
bpm = 130
bars = 4
barpm = 130/4

#____________kick___________________
#calibrates the kick timing with the bpm and the number of kicks per bar
kpb = 4 # kicks per bar
kpm = barpm*kpb # kicks per minute
kps = kpm/60 #kicks per second
ks = 1/kps #kick transient
kis = 12 #kicks in song

#fundamental frequency of kick
fk = 50
#transient time
T2 = .5
#time mapping
time_kick = np.linspace(0,T2,int(T2*nyquist))

#Volume envelope for kick
trans = np.linspace(1,0,int(T2*nyquist))
#pitch envelope for kick
trans_pitch = np.linspace(0,1,int(T2*nyquist))
intercept = .2
kick = 10*synthesizer.kick(fk, trans, trans_pitch, time_kick,intercept)
kick = synthesizer.wave_map(kick,nyquist)
kick_single = kick

#adds the kicks together to form a portion of the drum pattern.
for i in range(1,kis):
    kick = kick + sequencer.time_shift(kick_single,i*ks)
kick.write("kick.wav")

#_________Crash_________
#calibrates the crash with the bpm and the bars

cpb = 1 #crashs per bar
cpm = barpm*cpb
cps = cpm/60
cs = 1/cps #crashes in transient
cis = 1 #crashes i nsong

#fundamental frequency of the crash
fcrash = 100
#attack, decay time, decay amount, sustain, release
A =.001
D = .03
D_A = .5
S = .01
R = 1.2
#time cumulation of the volume envelope (as described above)
T2 = A+D+S+R
#time mapping
time = np.linspace(0,T2, int(T2*nyquist))

#frequency modulation table
fm =  [200,250,0,0,0,0]
#amplitude of modulating waves
Amp = [100,100,0,0,0,0]

#fm
crash = synthesizer.fm_(fcrash,fm,Amp,time)
#produces envelope
env_ = synthesizer.envelope(A,D,D_A,S,R, nyquist)
#applies envelope to sound
crash = crash*env_
#converts the numerical mapping into wave.
crash = synthesizer.wave_map(crash,nyquist)
crash_single = crash

#sequences the crash to produce beat
for i in range(1,cis):
    crash = crash + sequencer.time_shift(crash_single,i*cs)

#_________hats______________
hpb = np.asarray([4,2,4]) #hat per bar
hpm = barpm*hpb
hps = hpm/60
hs = 1/hps #hats in transient
his = [12,6,12] #hats in song
time_start = [.170,.240,.413] #initial starting time (otherwise they start at 0 sec.)


#since we have 3 different hats in this song, all of the parameters below have been expanded into a list containing 3 elements each
#fundamental frequency of hat
fhat = [100,500,1000]
# ADSR envelope specs
A = [.001,.001,.001]
D = [.03,.04,.02]
D_A = [.2,.1,.5]
S = [.01,.05,.001]
R = [.01,.03,.001]
#fm
fhatm = [[500,700,0,0,0,0],[100,200,0,0,0,0], [300,200,0,0,0,0]]
# ampltude of fm wave
Amp = [[100,100,0,0,0,0], [100,500,0,0,0,0], [200,400,0,0,0,0]]

# i references a different hat
for i in range(0,len(fhat)):
    T3 = A[i]+D[i]+S[i]+R[i]
    # time mapping for each hat
    time = np.linspace(0,T3, int(T3*nyquist))
    hat_single = synthesizer.fm_(fhat[i],fhatm[i],Amp[i],time)
    env_ = synthesizer.envelope(A[i],D[i],D_A[i],S[i],R[i], nyquist)
    hat_single = hat_single*env_
    hat_single = synthesizer.wave_map(hat_single,nyquist)
    hat = hat_single
    hat = sequencer.time_shift(hat,time_start[i])
    #sequences the hat
    for j in range(1,his[i]):
        hat = hat + sequencer.time_shift(hat_single,j*hs[i]+time_start[i])
    if i == 0:
        hat_combo = hat
    else:
        hat_combo = hat_combo + hat

song_2 = kick + crash + hat_combo



#writes song
song_2.write("song_2.wav")

#downsampling
sound = AudioSegment.from_file("song_2.wav", format='wav', frame_rate=22050)
sound = sound.set_frame_rate(22050)
sound.export("song_2.wav", format='wav')

#___________supersaw?____________________________________________

fsaw = 100
x2_ = 20
T = 1
voices = 4
detune = 5
time = sequencer.time_np(T,nyquist)

for i in range(0,x2_):
    if i == 0:
        fs = synthesizer.sin_create(fsaw,time)
    else:
        fsaw = fsaw*2
        fs = fs + synthesizer.sin_create(fsaw, time)

saw = synthesizer.wave_map(fs, nyquist)
saw.write("saw.wav")

#___________Extra_test_HAT____________________________

fhat = 10000
T3 = 1
# frequency modulation 
fm = [250,3,0,0,0,0]
# Amplitude of fm waves
A = [100,20,0,0,0,0]
time = np.linspace(0,T3, int(T3*nyquist))

hat = 1/10*synthesizer.fm_(fhat,fm,A,time)
hat = synthesizer.wave_map(hat,nyquist)
hat.write("hat.wav")

#_________Bell_______________

#list of frequency to be added at least needed to create the bell noise
f = [250,790,1310, 1829,2356,2914,3426]
#attack decay sustain release
A =.001
D = .2
D_A = .5
S = .001
R = 1
T = A+D+S+R

time = sequencer.time_np(T,nyquist)
#ads each freuency element cumulatively 
for i in range(0,len(f)):
    bell_single = synthesizer.sin_create(f[i],time)
    if i == 0:
        bell = bell_single
    else:
        bell = bell + bell_single

bell = bell*synthesizer.envelope(A,D,D_A,S,R,nyquist)
bell = synthesizer.wave_map(bell, nyquist)
bell.write("bell.wav")


C:\Users\Jason\anaconda3\lib\site-packages\thinkdsp.py:690: UserWarning: Can't add these waveforms; their time arrays don't line up.
  warnings.warn("Can't add these waveforms; their "


Writing kick.wav
Writing song_2.wav
Writing saw.wav
Writing hat.wav
Writing bell.wav
